In [1]:
# python notebook for Make Your Own Neural Network
# code for a 3-layer neural network, and code for learning the MNIST dataset
# this version trains using the MNIST dataset, then tests on our own images
# (c) Tariq Rashid, 2016
# license is GPLv2

In [2]:
import numpy
# scipy.special for the sigmoid function expit()
import scipy.special
# library for plotting arrays
import matplotlib.pyplot
# ensure the plots are inside this notebook, not an external window
%matplotlib inline

In [3]:
import os

In [4]:
# helper to load data from PNG image files
import imageio

In [5]:
# neural network class definition
class neuralNetwork2HiddenLayers:
    
    
    # initialise the neural network
    def __init__(self, inputnodes, hiddennodesLayer1, hiddennodesLayer2, outputnodes, learningrate):
        # set number of nodes in each input, hidden, output layer
        self.inodes = inputnodes
        self.hnodesLayer1 = hiddennodesLayer1
        self.hnodesLayer2 = hiddennodesLayer2
        self.onodes = outputnodes
        
        # link weight matrices, wih and who
        # weights inside the arrays are w_i_j, where link is from node i to node j in the next layer
        # w11 w21
        # w12 w22 etc 
        self.wih1 = numpy.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodesLayer1, self.inodes))
        self.wh1h2 = numpy.random.normal(0.0, pow(self.hnodesLayer1, -0.5), (self.hnodesLayer2, self.hnodesLayer1))
        self.wh2o = numpy.random.normal(0.0, pow(self.hnodesLayer2, -0.5), (self.onodes, self.hnodesLayer2))

        # learning rate
        self.lr = learningrate
        
        # activation function is the sigmoid function
        self.activation_function = lambda x: scipy.special.expit(x)
        
        pass

    
    # train the neural network
    def train(self, inputs_list, targets_list):
   # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        
        # calculate signals into hidden layer 1
        hidden_inputs_l1 = numpy.dot(self.wih1, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs_l1 = self.activation_function(hidden_inputs_l1)

        # calculate signals into hidden layer 2
        hidden_inputs_l2 = numpy.dot(self.wh1h2, hidden_outputs_l1)
        # calculate the signals emerging from hidden layer
        hidden_outputs_l2 = self.activation_function(hidden_inputs_l2)
 
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.wh2o, hidden_outputs_l2)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        # output layer error is the (target - actual)
        output_errors = targets - final_outputs
        # hidden layer 2 error is the output_errors, split by weights, recombined at hidden nodes
        hidden_errors_l2 = numpy.dot(self.wh2o.T, output_errors)
        # hidden layer 1 error is the output_errors, split by weights, recombined at hidden nodes
        hidden_errors_l1 = numpy.dot(self.wh1h2.T, hidden_errors_l2)
        
        # update the weights for the links between the hidden layer 2 and output layers
        self.wh2o += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden_outputs_l2))

        # update the weights for the links between the input and hidden layer l1
        self.wh1h2 += self.lr * numpy.dot((hidden_errors_l2 * hidden_outputs_l2 * (1.0 - hidden_outputs_l2)), numpy.transpose(hidden_outputs_l1))
        
        # update the weights for the links between the input and hidden layer l1
        self.wih1 += self.lr * numpy.dot((hidden_errors_l1 * hidden_outputs_l1 * (1.0 - hidden_outputs_l1)), numpy.transpose(inputs))
        
        pass

    
    # query the neural network
    def query(self, inputs_list):
        inputs = numpy.array(inputs_list, ndmin=2).T
        
        # calculate signals into hidden layer 1
        hidden_inputs_l1 = numpy.dot(self.wih1, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs_l1 = self.activation_function(hidden_inputs_l1)

        # calculate signals into hidden layer 2
        hidden_inputs_l2 = numpy.dot(self.wh1h2, hidden_outputs_l1)
        # calculate the signals emerging from hidden layer
        hidden_outputs_l2 = self.activation_function(hidden_inputs_l2)
 
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.wh2o, hidden_outputs_l2)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs

In [6]:
# neural network class definition
class neuralNetwork1HiddenLayer:
    
    
    # initialise the neural network
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        # set number of nodes in each input, hidden, output layer
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        
        # link weight matrices, wih and who
        # weights inside the arrays are w_i_j, where link is from node i to node j in the next layer
        # w11 w21
        # w12 w22 etc 
        self.wih = numpy.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodes, self.inodes))
        self.who = numpy.random.normal(0.0, pow(self.hnodes, -0.5), (self.onodes, self.hnodes))

        #dim1 = len(self.wih)
        #dim2 = len(self.wih[0])
        #print(dim1)
        #print(dim2)
        
        # learning rate
        self.lr = learningrate
        
        # activation function is the sigmoid function
        self.activation_function = lambda x: scipy.special.expit(x)
        
        pass

    
    # train the neural network
    def train(self, inputs_list, targets_list):
        # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        # output layer error is the (target - actual)
        output_errors = targets - final_outputs
        # hidden layer error is the output_errors, split by weights, recombined at hidden nodes
        hidden_errors = numpy.dot(self.who.T, output_errors) 
        
        # update the weights for the links between the hidden and output layers
        self.who += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden_outputs))
        
        # update the weights for the links between the input and hidden layers
        self.wih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), numpy.transpose(inputs))
        
        pass

    
    # query the neural network
    def query(self, inputs_list):
        # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs

In [7]:
# number of input, hidden and output nodes
input_nodes = 9
hidden_nodes_l1 = 200
hidden_nodes_l2 = 9
output_nodes = 9

# learning rate
learning_rate = 0.1

# create instance of neural network
#n = neuralNetwork2HiddenLayers(input_nodes,hidden_nodes_l1,hidden_nodes_l2,output_nodes,learning_rate)
n = neuralNetwork1HiddenLayer(input_nodes,hidden_nodes_l1,output_nodes,learning_rate)

In [8]:
# load the mnist training data CSV file into a list
training_data_file = open("nnTrainingData.txt", 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

In [9]:
print("train the neural network")
# train the neural network

# epochs is the number of times the training data set is used for training
epochs = 100
for e in range(epochs):
    print("Epoch "+str(e)+"/"+str(epochs))
    for e in range(epochs):
        i=0
        for training_data in training_data_list:            
            if i % 2 == 0:
                inputVector = numpy.asfarray(training_data.replace("\n", "").split(',')) 
            else:
                outputVector = numpy.asfarray(training_data.replace("\n", "").split(','))
                n.train(inputVector, outputVector)
            i=i+1
        pass        
pass    

train the neural network
Epoch 0/100
Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/1

In [10]:
print("test the neural network")
# test the neural network

equals = 0
unequals = 0
i=0
for training_data in training_data_list:
    if i % 2 == 0:
        inputVector = numpy.asfarray(training_data.replace("\n", "").split(',')) 
    else:
        outputVector = numpy.asfarray(training_data.replace("\n", "").split(','))
        resultVector = n.query(inputVector)

        maxIdxOutputVector = numpy.argmax(outputVector)
        maxIdxResultVector = numpy.argmax(resultVector)

        if (maxIdxOutputVector == maxIdxResultVector):
            equals = equals + 1
        else:
            unequals = unequals + 1
    i=i+1
pass

print("Equals : " + str(equals))
print("Unequals : " + str(unequals))

test the neural network
Equals : 2993
Unequals : 1527
